<a href="https://colab.research.google.com/github/Rhqo/LLM_Study/blob/main/4_fine_tuned_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install openai -q
!pip install --upgrade pyarrow
!pip install datasets -q
!pip install tqdm -q
!pip install tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.7 MB/s eta 0:00:00


In [57]:
import pandas as pd
from openai import OpenAI
import os
from datasets import load_dataset, DatasetDict
import random
import math
from google.colab import userdata
import json
import tiktoken
import numpy

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

random.seed(42)


# Load Datasets

In [5]:
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Magazine_Subscriptions", trust_remote_code=True)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating full split: 0 examples [00:00, ? examples/s]

DatasetDict({
    full: Dataset({
        features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
        num_rows: 71497
    })
})

In [6]:
train_test_ratio = 0.8
full_dataset = dataset['full']

split_dataset = full_dataset.train_test_split(test_size=1 - train_test_ratio, seed=42)

# train과 validation 데이터셋 확인
train_dataset = split_dataset['train']
validation_dataset = split_dataset['test']

training_df = pd.DataFrame(train_dataset)
validation_df = pd.DataFrame(validation_dataset)

print(training_df.head())
print(validation_df.head())

   rating                                              title  \
0     1.0        Magazine is great... but you won't get them   
1     5.0                                         Five Stars   
2     5.0                                         Five Stars   
3     3.0                                        Three Stars   
4     2.0  But a girl can dream of having a home like Mar...   

                                                text images        asin  \
0  The company that distributes this magazine in ...     []  B000FDDYH8   
1               received today thanks great magazine     []  B0000AN46X   
2  Every gardener with dreams should take this ma...     []  B0045FEHD2   
3                         Not sure if I will reorder     []  B004GVZUU4   
4  Have had subscriptions to this magazine many t...     []  B002PXW0EO   

  parent_asin                       user_id      timestamp  helpful_vote  \
0  B000FDDYH8  AHI4BQKYERYXRSHGRB7DYIKOBLUQ  1310487605000            10   
1  B0000AN46

In [7]:
training_df[['title', 'text', 'rating']].sample(5)

,title,text,rating
44149,FlyTyer is a supporter of Tucker Carlson. I am...,"If you love Fox News and Tucker Carlson, buy t...",1.0
21949,"Self-sufficiency for economizers, preppers, he...",What a great magazine. The world is now a bett...,5.0
34450,One Star,never subscribed to this magazine,1.0
8694,👎,👎,3.0
47528,Best Magazine You Can Buy,Reader's Digest is such a great magazine. Time...,5.0


In [35]:
def prepare_df_for_openai(df):
    df['prompt'] = df['title'] + '\n\n' + df['text'] + '\n\n###\n\n'
    df['completion'] = ' ' + df['rating'].astype(str)
    result_df = df
    result_df.drop_duplicates(subset=['prompt'], inplace=True)
    return result_df[['prompt', 'completion']].sample(len(result_df))

In [36]:
train_df = prepare_df_for_openai(training_df)
val_df = prepare_df_for_openai(validation_df)

In [37]:
train_df.to_json("amazon-train.jsonl", orient='records', lines=True)
val_df.to_json("amazon-val.jsonl", orient='records', lines=True)

In [73]:
data_path = "/content/amazon-train.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[:1]:
    print(message)

Num examples: 54257
First example:
{'prompt': "Can't  discontinue subscription\n\nBe careful.  My subscription was renewed without notice and when I tried to cancel it after the fact I was refused.\n\n###\n\n", 'completion': ' 1.0'}


### Start Fine-tuning

In [77]:
train_response = client.files.create(
    file=open("/content/amazon-train.jsonl", "rb"),
    purpose='fine-tune'
)

val_response = client.files.create(
    file=open("amazon-val.jsonl", "rb"),
    purpose='fine-tune'
)

print(f"Uploaded training file: {train_response.id}")

Uploaded training file: file-rfpuMba9gQZ43xxrlOeK35al


In [86]:
client.fine_tuning.jobs.create(
  training_file=train_response.id,
  model="gpt-3.5-turbo"
)

BadRequestError: Error code: 400 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

In [30]:
# List 10 fine-tuning jobs
#client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-AsvBBKHHgnbtDuaC1UgUCvlu")

# # Cancel a job
# client.fine_tuning.jobs.cancel("ftjob-abc123")

# # List up to 10 events from a fine-tuning job
# client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

FineTuningJob(id='ftjob-AsvBBKHHgnbtDuaC1UgUCvlu', created_at=1719813896, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Invalid file format. Input file file-ELd3LRX4Gg0V5tDIkcGybyVM is in the prompt-completion format, but the specified model gpt-3.5-turbo-0125 is a chat model and requires chat-formatted data. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for details.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-IGE1n4VtYEKHaRRX86nVPpRS', result_files=[], seed=270480675, status='failed', trained_tokens=None, training_file='file-ELd3LRX4Gg0V5tDIkcGybyVM', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [94]:
# question = "hi"

# response = client.completions.create(
#     model=response.id,
#     prompt=question,
#     # max_tokens=100
# )

# # 응답 출력
# answer = response.choices[0].text.strip()
# print(f"Question: {question}")
# print(f"Answer: {answer}")

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:test-key::9g55WyPO",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "can you give me what you learned in fine-tuning process?"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='In the fine-tuning process, the model is trained on a specific dataset to improve performance on a particular task.', role='assistant', function_call=None, tool_calls=None)
